In [11]:
# importing the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
123yuvraj


ValueError: ignored

In [ ]:
# Reading the input
data = pd.read_csv('/content/drive/MyDrive/pubg.csv')
data.head()

In [ ]:
data.dropna(inplace=True)
print(data.shape)

# Outlaiers Detection

- Kills without moving is not possible in game so lets check if there are any records with zero moment and have kills

In [ ]:
# Create feature totalDistance
data['totalDistance'] = data['rideDistance'] + data['walkDistance'] + data['swimDistance']
# Create feature killsWithoutMoving
data['killsWithoutMoving'] = ((data['kills'] > 0) & (data['totalDistance'] == 0))

In [ ]:
# See wheather there are players with kills and no movement
display(data[data['killsWithoutMoving'] == True].shape)
data[data['killsWithoutMoving'] == True].head()

In [ ]:
# Remove these data's as they are invalid or outlaiers
data.drop(data[data['killsWithoutMoving'] == True].index, inplace=True)

In [ ]:
data.shape

In [ ]:
# Players with more road kills in a single match is not most likely to happen lets check for those data
r_kill = data[data['roadKills'] > 10]
print(r_kill.shape)
r_kill

- looks like this data is ok.
- but player with id c3e444f7d1289f has ride distance 5m and road kills 14 so its naturally not possible 

In [ ]:
#creating new feature r_kill to find outlaiers
data['r_kill'] = ((data['roadKills'] > 10) & (data['rideDistance'] < 0.100))
print(data[data['r_kill'] == True].shape)
data[data['r_kill']==True].head()

In [ ]:
# removing the outlaier in road kills
data.drop(data[data['r_kill'] == True].index, inplace=True)
data.shape

- plotting data with kill counts

In [ ]:
sns.catplot(x=data['kills'],data=data,aspect=3,kind='count')

- have a closer look above max kill counts a player has is around 15.
- but players more than that can be considered as outlaiers so we set limit to 30 kills a player can have.

In [ ]:
# Players with more than 30 kills
display(data[data['kills'] > 30].shape)
data[data['kills'] > 30].head(10)

In [ ]:
# removing these outlaiers
data.drop(data[data['kills'] > 30].index, inplace = True)
data.shape

In [ ]:
# Create headshot_rate feature
data['headshot_rate'] = data['headshotKills'] / data['kills']
data['headshot_rate'] = data['headshot_rate'].fillna(0)

In [ ]:
print(data.shape)
data.head()

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(data['headshot_rate'])
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data['headshot_rate'])
plt.show()

In [ ]:
# Players who have head shot kills 100% means, kills = head shot kills
print(data[(data['headshot_rate'] == 1) & (data['kills'] > 9)].shape)
data[(data['headshot_rate'] == 1) & (data['kills'] > 9)].head()

- some players might be good players and have all head shots so lets keep these data

In [ ]:
# Plot of longestKill
plt.figure(figsize=(12,4))
sns.distplot(data['longestKill'])
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data['longestKill'])
plt.show()

In [ ]:
# players with kills with distance more than 1 km
print(data[data['longestKill'] >= 1000].shape)
data[data['longestKill'] >= 1000].head()

- I think more than 1000mt is highly outlaiered so removing them

In [ ]:
data.drop(data[data['longestKill'] >= 1000].index, inplace = True)
data.shape

In [ ]:
# Plot of the weaponsAcquired
plt.figure(figsize=(12,4))
sns.distplot(data['weaponsAcquired'])
plt.show()

In [ ]:
# Plot of the weaponsAcquired
plt.figure(figsize=(12,4))
sns.boxplot(data['weaponsAcquired'])
plt.show()

- from above graph its clear that most of players have less than 20 weapons acquired so we can remove outlaiers in this
- even thouhg after 20 all are outlaiers and as its dense till 50, we set limit 45 and lets see data

In [ ]:
# Players who acquired more than 45 weapons
print(data[data['weaponsAcquired'] >= 45].shape)
data[data['weaponsAcquired'] >= 45].head()

In [ ]:
# dropping players with weapon acquired more than 45
data.drop(data[data['weaponsAcquired'] >= 45].index, inplace = True)
data.shape

In [ ]:
# plot of heals used by players
plt.figure(figsize=(12,4))
sns.distplot(data['heals'])
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data['heals'])
plt.show()

- According to above graphs heals used by most of the players is 5 so above that are outlaiers, we set limit to 35

In [ ]:
print(data[data['heals'] >= 35].shape)
data[data['heals'] >= 35].head()

In [ ]:
# dropping the outlaiers in heals
data.drop(data[data['heals'] >= 35].index, inplace = True)
data.shape

In [ ]:
# Encoding the match type feature
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
data.matchType = enc.fit_transform(data.matchType)
data.matchType.unique()

- Categorical encoding on match id and group id so they help in decission tree of Random Forest

In [ ]:
# Turn groupId and match Id into categorical types
data['groupId'] = data['groupId'].astype('category')
data['matchId'] = data['matchId'].astype('category')

# Get category coding for groupId and matchID
data['groupId_cat'] = data['groupId'].cat.codes
data['matchId_cat'] = data['matchId'].cat.codes

# Get rid of old columns
data.drop(columns=['groupId', 'matchId'], inplace=True)

# Lets take a look at our newly created features
data[['groupId_cat', 'matchId_cat']].head()

In [ ]:
# dropping ID column as not required
data.drop(columns=['Id'],axis=1,inplace=True)

In [ ]:
# Importing necessary package for modeling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [ ]:
data.corrwith(data['winPlacePerc'])

In [ ]:
# Take sample for easy modeling and understanding
sample = 1050000
data_sample = data.sample(sample)

In [ ]:
# splitting up of data
X = data_sample.loc[ : ,['assists','boosts','damageDealt','DBNOs','headshotKills','heals','killPlace', 'killPoints','kills',
                  'killStreaks','longestKill','matchDuration', 'matchType','maxPlace','numGroups','rankPoints','swimDistance'
                  ,'revives','roadKills', 'teamKills','vehicleDestroys','weaponsAcquired','winPoints','walkDistance','rideDistance']] 
y = data_sample.loc[ : ,['winPlacePerc']] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.shape,X_test.shape

# Random Forest

In [ ]:
model = RandomForestRegressor()
model.fit(X_train,y_train)
y_predict=model.predict(X_test)
print(r2_score(y_test,y_predict))
print(mean_absolute_error(y_test,y_predict))
print(mean_squared_error(y_test,y_predict))

In [ ]:
model = RandomForestRegressor(n_estimators=100,min_samples_leaf=7)
model.fit(X_train,y_train)
y_predict=model.predict(X_test)
print(r2_score(y_test,y_predict))
print(mean_absolute_error(y_test,y_predict))
print(mean_squared_error(y_test,y_predict))

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
y_predict=model.predict(X_test)
print(r2_score(y_test,y_predict))
print(mean_absolute_error(y_test,y_predict))
print(mean_squared_error(y_test,y_predict))

In [ ]:
ax1=sns.distplot(y_test,hist=False,color='red',label="actual value")
sns.distplot(y_predict,hist=False,color='b',label="fitted value")
plt.legend(loc='upper right')

# XGBoost

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X_train,y_train)
y_predict=model.predict(X_test)
print(r2_score(y_test,y_predict))
print(mean_absolute_error(y_test,y_predict))
print(mean_squared_error(y_test,y_predict))

# ANN

In [ ]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
print(r2_score(y_test,y_predict))
print(mean_absolute_error(y_test,y_predict))
print(mean_squared_error(y_test,y_predict))

In [ ]:
model = MLPRegressor(hidden_layer_sizes=(100,50,30))
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
print(r2_score(y_test,y_predict))
print(mean_absolute_error(y_test,y_predict))
print(mean_squared_error(y_test,y_predict))

# CNN

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
# Data Modeling
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, input_shape = [25], activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'linear')  
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
model.summary()
model.fit(X_train, y_train, epochs=10)

In [ ]:
train_predict = model.predict(X_train)
train_msq = mean_squared_error(y_train, train_predict)
train_r2 = r2_score(y_train, train_predict)
print("Train:\nMean Squared Error :", train_msq, "\nR2 score :", train_r2)

test_predict = model.predict(X_test) 
test_msq = mean_squared_error(y_test, test_predict) 
test_r2 = r2_score(y_test, test_predict) 
print("\nTest:\nMean Squared Error :", test_msq, "\nR2 score :", test_r2)